# Example Capacity Test using pvcaptest
This example goes through typical steps of performing a capacity test following the ASTM E2848 standard using the pvcaptest package.

## Imports

In [ ]:
%matplotlib inline

import pandas as pd

# import captest as pvc
import captest as ct
from captest import capdata as pvc
from bokeh.io import output_notebook, show

# uncomment below two lines to use cptest.scatter_hv in notebook
import holoviews as hv
hv.extension('bokeh')

#if working offline with the CapData.plot() method may fail
#run 'export BOKEH_RESOURCES=inline' at the command line before
#running the jupyter notebook

output_notebook()

## Load and Plot Measured Data

We begin by using the `load_data` function, which reads the file(s) specified by the `path` argument and returns an instance of the `CapData` class.  In this example we will calculate reporting conditions from the measured data, so we load and filter the measured data first.

When given the path to a file, as shown here, `load_data` will try to read that file. If you pass a path to a directory, `load_data` will look for and attempt to load all files ending with '.csv' in the specified directory. Other file types can also be loaded by passing your own function to the `file_reader` argument and including the extension (e.g. 'xlsx') as a kwarg.

In [ ]:
das = ct.load_data('./data/example_measured_data.csv')

The `load_data` method loads the data into a pandas DataFrame, which it assigns to the `data` attribute of the `CapData` object.  Here we use the pandas DataFrame `head` method to return the first three rows.

In [ ]:
das.data.head(3)

In addition to loading data, by default the `load_data` function attempts to parse the column headers and group the columns based on the type of measurement recorded in each column.  For each inferred measurement type, `group_columns` creates an abbreviated name and a list of columns that contain measurements of that type. The python dictionary created by `group_columns` is stored in the `column_groups.data` attribute. `column_groups` is a dictionary that display nicely and includes the groups as attibutes for easy access as shown below. If the column grouping returned is not correct, you can provide either your own function to group the columns or a yaml, json, or excel file mapping column group identifiers to the column headings.

In [ ]:
das.column_groups

When working in an environment with table completion, like a Jupyter notebook, the group attributes are easy to access without needing to remember the group name.

In [ ]:
das.column_groups.irr_poa_pyran

The CapData has the methods`loc` and `floc` to select subsets of columns from the `data` and `data_filtered` DataFrames, respectively. These methods allow easy access to the groups of columns identified in `column_groups` using the `column_group` keys, column names, `regression_cols` keys, or a combination of the three. The `regression_cols` attribute is introduced below.  The `column_groups` dictionary also enables much of the functionality of `CapData` methods to perform common capacity testing tasks, like generating scatter plots, filtering data, and performing regressions.

Using the `loc` method with the 'irr_poa_ref_cell' attribute key of `column_groups` to select data from the POA reference cell columns in the `data` DataFrame:

In [ ]:
das.loc[das.column_groups.irr_poa_ref_cell].iloc[100:103, :]

Accessing the two irradiance columns of the 'irr_poa_pyran' group, the single column of the '_mtr_' group, and the 'met1_amb_temp' column of the `data_filtered` DataFrame:

In [ ]:
das.floc[['irr_poa_pyran', '_mtr_', 'met1_amb_temp']]

pvcaptest does not attempt to determine which columns of data or groups of columns are the data to be used in the regressions. The link between regression variables and the imported data is made by a dictionary stored in the `regression_cols` attribute.  pvcaptest provides the convience method `set_regression_cols` for this purpose. `regression_cols` should be set immediately after loading data as many other `CapData` methods rely on this attribute.

In [ ]:
das.set_regression_cols(power='_mtr_', poa='irr_poa_pyran', t_amb='temp_amb_', w_vel='wind__')

Once the regression columns are set, the `loc` or `floc` methods will return the data for each type of sensor identified in the `column_groups` attribute.

Here we are accessing the same POA irradiance data as above with `loc` and the group name, but now using the regression variable id.

In [ ]:
das.loc['poa'].iloc[100:103, :]

For datasets that have multiple measurements of the same value, like the two POA irradiance measurements in this sample data, these values must be aggregated prior to filtering or regressing the data.  The `agg_sensors` method provides a convient way to do this for all the groups of measurements in `column_groups` in one step.

The desired aggregations are specified by passing a dictionary to the `agg_map` argument where the keys are groups from `column_groups` and the values are aggregation functions.  Here we are using string functions that are recognized by pandas.  Most of the common aggregation functions (mean, median, max, sum, min, etc.) are available as string functions.  If you would like to apply a different aggregation function, please refer to the pandas documentation for `DataFrame.agg`. By default, the `agg_sensors` method adds a new column to the dataframe in the `data` attribute for the results of each aggregation and copies over the `data_filtered` attribute with the new dataframe.

There is a also a method, `filter_sensors`, for filtering data on comparisons between measurements of the same value described below.

In [ ]:
das.agg_sensors(agg_map={'_inv_':'sum', 'irr_poa_pyran':'mean', 'temp_amb_':'mean', 'wind__':'mean'})

The plot method creates a dashboard with a group of time series plots that are useful for visually inspecting the imported data.

`plot` uses the structure of the `column_groups` attribute to create a layout of plots. A single plot is generated for each measurement type and each column with measurements of that type are plotted as a separate line on the plot. In this example there are two different weather stations, which each have pyranometers measuring plane of array and global horizontal irradiance. This arrangement of sensors results in two plots which each have two lines.

Note, the full functionality of the dashboard requires a live notebook. Try installing to run or using the launch binder button at the top of the page. 

In [ ]:
combine = {'inv_sum_mtr_pwr': ['mtr', 'inv.*agg'], 'irr_all':['irr_poa', 'irr_ghi'], 'temp_all':['temp_amb', 'temp_mod']}
default_groups = ['inv_sum_mtr_pwr', 'irr_all', 'temp_all']
das.plot(combine=combine, default_groups=default_groups)

## Filtering Measured Data
The `CapData` class provides a number of convience methods to apply filtering steps as defined in ASTM E2848.  The following section demonstrates the use of the more commonly used filtering steps to remove measured data points.

In [ ]:
# Uncomment and run to copy over the filtered dataset with the unfiltered data.
das.reset_filter()

A common first step is to review the scatter plot of the POA irradiance against the power production.  

If you have the optional dependency Holoviews installed, `scatter_hv` will return an interactive scatter plot.  Additionally, `scatter_hv` includes an option to return a timeseries plot of power that is linked to the scatter plot, so points selected in the scatter plot will be highlighted in the time series.

In [ ]:
# Uncomment the below line to use scatter_hv with linked time series
das.scatter_hv(timeseries=True)

In this example, we have multiple measurements of the same value from different sensors.  In this case a common first step is to compare measurements from the different sensors and remove data for timestamps where the measurements differ above some acceptable threshold.  The `filter_sensors` method provides a convient method to accomplish this taks for the groups of measurements identified as regression values.

In [ ]:
das.filter_sensors()

The `get_summary` method will return a dataframe summarizing the filtering steps that have been applied, the agruments passed to them, the number of points prior to filtering, and the number of points after filtering.

In [ ]:
das.get_summary()

The `filter_custom` method provides a way to use your own filtering method within captest and update the summary data.  The `filter_custom` method allows passing any function or method that takes a DataFrame as the first argument and returns a filtered dataframe with rows removed.  Passed methods can be user-defined functions or Pandas DataFrame methods.

Below, we use the `filter_custom` method with the pandas DataFrame `dropna` method to removing missing data and update the summary data.

In [ ]:
das.filter_custom(pd.DataFrame.dropna)

The `filter_irr` method provides a convient way to remove remove data based on the irradiance measurments.  Here we use it to remove periods of low irradiance. Values greater than 2000 W/m<sup>2</sup> will also be removed, if present.

In [ ]:
das.get_summary()

In [ ]:
das.filter_irr(200, 2000)

We can re-run the `scatter` method to see the results of the filtering steps.

In [ ]:
das.scatter_hv()

The `filter_outliers` method uses scikit-learn's elliptic envelope to remove outlier points.  A future release will include a way to interactively select points to be removed.

In [ ]:
das.filter_outliers()

In [ ]:
das.scatter_hv()

The `fit_regression` method performs a regression on the data stored in `data_filtered` using the regression equation specified by the standard.  The regression equation is stored in the `regression_formula` attribute as shown below.  Regressions are performed using the statsmodels package.

Below, we set the filter argument of the `fit_regression` method to `True` to remove time periods when the residual exceeds two standard deviations of the mean residual.

In [ ]:
das.regression_formula

In [ ]:
das.fit_regression(filter=True, summary=False)

In [ ]:
das.get_summary()

____
#### Calculation of Reporting Conditions

The `rep_cond` method provide a variety of ways to calculate reporting conditions.  Using `rep_cond` the reporting conditions are always calculated from the data store in the df_flt attribute.  Refer to the example notebook "Reporting Conditions Examples" for a thourough explanation of the `rep_cond` functionality.  By default the reporting conditions are calcualted following the guidance of ASTM E2939-13.

In [ ]:
das.rep_cond()

----

Previously we used the irradiance filter to filter out data below 200 W/m<sup>2</sup>.  The irradiance filter can also be used to filter irradiance based on a percentage band around a reference value.  This approach is shown here to remove data where the irradiance is outside of +/- 50% of the reporting irradiance.

In [ ]:
das.filter_irr(0.5, 1.5, ref_val=das.rc['poa'][0])

In [ ]:
das.scatter_hv()

The `fit_regression` method is used again with the default arguments, which result in fitting the regression, printing and storing the results, but not filtering. The result of the regression is a statsmodels `RegressionResultsWrapper` object containing the regression coefficients and other information generated when performing the regression.  This object is stored in the CapData `regression_results` attribute.

In [ ]:
das.fit_regression()

The regression coefficients and p-values for each term are attributes available in the `regression_results`.

In [ ]:
das.regression_results.params

In [ ]:
das.regression_results.pvalues

## Load and Filter PVsyst Data

To load and filter the modeled data, often from PVsyst, we use the `load_pvsyst` method, which returns a CapData object with the pvsyst data loaded

In [ ]:
sim = ct.load_pvsyst('./data/pvsyst_example_HourlyRes_2.CSV')

In [ ]:
sim.column_groups

In [ ]:
sim.set_regression_cols(power='real_pwr__', poa='irr_poa_', t_amb='temp_amb_', w_vel='wind__')

In [ ]:
# sim.plot()

In [ ]:
# Write over cptest.flt_sim dataframe with a copy of the original unfiltered dataframe
sim.reset_filter()

As a first step we use the `filter_time` method to select a 60 day period of data centered around the measured data.

In [ ]:
sim.filter_time(test_date='10/11/1990', days=60)

In [ ]:
sim.scatter_hv()

In [ ]:
sim.filter_irr(200, 930)

In [ ]:
sim.scatter_hv()

In [ ]:
sim.get_summary()

The `filter_pvsyt` method removes data for times when shade is present or the `IL Pmin`, `IL Vmin`, `IL Pmax`, `IL Vmax` output values are greater than 0.

In [ ]:
sim.filter_pvsyst()

In [ ]:
sim.filter_irr(0.5, 1.5, ref_val=das.rc['poa'][0])

In [ ]:
sim.fit_regression()

## Results

The `get_summary` and `captest_results_check_pvalues` functions display the results of filtering on simulated and measured data and the final capacity test results comparing measured capacity to expected capacity, respectively.

In [ ]:
pvc.get_summary(das, sim)

In [ ]:
pvc.captest_results_check_pvalues(sim, das, 6000, '+/- 7', print_res=True)

Uncomment and run the below lines to produce a scatter plot overlaying the final measured and PVsyst data.

In [ ]:
%%opts Scatter (alpha=0.3)
%%opts Scatter [width=600]
das.scatter_hv().relabel('Measured') * sim.scatter_hv().relabel('PVsyst')